<center>Myranda Swartzwelter<br>
    DSC-550 T301 Data Mining<br>
    Week 3 Assignment: NLP<br>
 3/29/22
   

1. Import the movie review data as a data frame and ensure that the data is loaded properly.

In [100]:
#import necessary packages
import pandas as pd
import numpy as np
from textblob import TextBlob 

In [102]:
#import file note it is a tsv
reviews_train = pd.read_csv('labeledTrainData.tsv', sep='\t')

#confirm that data is properly loaded - should have 25000 rows and looking at the first few rows should show ID, sentiment and review
reviews_train.head()
reviews_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


2. How many of each positive and negative reviews are there?

In [103]:
#count the number of rows grouping by sentiment
reviews_train.groupby(by='sentiment')['id'].count()

#there are an equal number of positive and negative reviews - 12500 of each

sentiment
0    12500
1    12500
Name: id, dtype: int64

3. Use TextBlob to classify each movie review as positive or negative. Assume that a polarity score greater than or equal to zero is a positive sentiment and less than 0 is a negative sentiment.



In [104]:
#create an array to list the sentiments we generate
sentiments = []

#iterate through dataframe and get the polarity for each review
for index, row in reviews_train.iterrows():
    blob = TextBlob(row['review'])
    polarity = blob.sentiment.polarity
    
    #if the polarity score >= 0 this is a positive sentiment and <0 is a negative sentiment
    if polarity < 0:
        sentiment = 0
    else:
        sentiment = 1
    #add sentiment to list
    sentiments.append(sentiment)

#create a new colum with the modeled sentiments
reviews_train['modeled_sentiments'] = sentiments
reviews_train.head()

,id,sentiment,review,modeled_sentiments
0,5814_8,1,With all this stuff going down at the moment w...,1
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...",1
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,0
3,3630_4,0,It must be assumed that those who praised this...,1
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,0


4. Check the accuracy of this model. Is this model better than random guessing?

In [105]:
#create a new column for values that are correct
reviews_train['correct'] = np.where(reviews_train['sentiment']== reviews_train['modeled_sentiments'], 1, 0)

#count number of correct by model
total_correct = reviews_train['correct'].sum()

#number of reviews
total = reviews_train['correct'].count()

#calculate accuracy
accuracy = total_correct / total *100
print(total_correct, total, accuracy)

#The model is better than a 50-50 guessing as 68>50

17131 25000 68.524


5. For up to five points extra credit, use another prebuilt text sentiment analyzer, e.g., VADER, and repeat steps (3) and (4).

In [107]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#create an array to list the sentiments we generate
sentiments_vader = []
sid_obj = SentimentIntensityAnalyzer()

#iterate through dataframe and get the polarity for each review
for index, row in reviews_train.iterrows():
    review = row['review']
    compound = sid_obj.polarity_scores(review)['compound']
    
    #if the polarity score >= 0 this is a positive sentiment and <0 is a negative sentiment
    #Not accounting for neutral (compound = 0)
    if compound < 0:
        sentiment = 0
    else:
        sentiment = 1
    #add sentiment to list
    sentiments_vader.append(sentiment)

#create a new column with the modeled sentiments
reviews_train['vader_sentiments'] = sentiments_vader
reviews_train.head()

,id,sentiment,review,modeled_sentiments,correct,vader_sentiments
0,5814_8,1,With all this stuff going down at the moment w...,1,1,0
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...",1,1,1
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,0,1,0
3,3630_4,0,It must be assumed that those who praised this...,1,0,0
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,0,0,1


In [108]:
#create a new column for values that are correct
reviews_train['correct_vader'] = np.where(reviews_train['sentiment']== reviews_train['vader_sentiments'], 1, 0)

#count number of correct by model
total_correct = reviews_train['correct_vader'].sum()

#number of reviews
total = reviews_train['correct_vader'].count()

#calculate accuracy
accuracy = total_correct / total *100
print(total_correct, total, accuracy)

17351 25000 69.404


Part 2 <br>
1. Convert all text to lowercase letters.

In [109]:
#convert to lowercase
reviews_train['review'] = reviews_train['review'].apply(lambda x: x.lower())

#check 
reviews_train.head()

,id,sentiment,review,modeled_sentiments,correct,vader_sentiments,correct_vader
0,5814_8,1,with all this stuff going down at the moment w...,1,1,0,0
1,2381_9,1,"\the classic war of the worlds\"" by timothy hi...",1,1,1,1
2,7759_3,0,the film starts with a manager (nicholas bell)...,0,1,0,1
3,3630_4,0,it must be assumed that those who praised this...,1,0,0,1
4,9495_8,1,superbly trashy and wondrously unpretentious 8...,0,0,1,1


2. Remove punctuation and special characters from the text.

In [110]:
#load library
import string

#create function to remove punctuation
def remove_punctuation(text):
    for punctuation in string.punctuation:
            text = text.replace(punctuation, ' ')
    return text
            
#apply function
reviews_train['review']= reviews_train['review'].apply(lambda x: remove_punctuation(x))

#check work
reviews_train.head()


,id,sentiment,review,modeled_sentiments,correct,vader_sentiments,correct_vader
0,5814_8,1,with all this stuff going down at the moment w...,1,1,0,0
1,2381_9,1,the classic war of the worlds by timothy hi...,1,1,1,1
2,7759_3,0,the film starts with a manager nicholas bell ...,0,1,0,1
3,3630_4,0,it must be assumed that those who praised this...,1,0,0,1
4,9495_8,1,superbly trashy and wondrously unpretentious 8...,0,0,1,1


3. Remove Stop Words

In [112]:
#load library
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


#download stopwords and punkt if needed
#import nltk
#nltk.download('stopwords')
#nltk.download('punkt')

In [113]:
#first tokenize the words in the reviews
reviews_train['tokenized_reviews'] = reviews_train['review'].apply(lambda x: word_tokenize(x))

#reviews_train.head()


In [114]:
#create stop words list
stop_words = stopwords.words('english')

#create function to remove stopwords
def remove_stopwords(text):
    text = [word for word in text if word not in stop_words]
    return text

#apply function to tokenized reviews
reviews_train['removed_stopwords'] = reviews_train['tokenized_reviews'].apply(lambda x: remove_stopwords(x))

reviews_train.head()

,id,sentiment,review,modeled_sentiments,correct,vader_sentiments,correct_vader,tokenized_reviews,removed_stopwords
0,5814_8,1,with all this stuff going down at the moment w...,1,1,0,0,"[with, all, this, stuff, going, down, at, the,...","[stuff, going, moment, mj, started, listening,..."
1,2381_9,1,the classic war of the worlds by timothy hi...,1,1,1,1,"[the, classic, war, of, the, worlds, by, timot...","[classic, war, worlds, timothy, hines, enterta..."
2,7759_3,0,the film starts with a manager nicholas bell ...,0,1,0,1,"[the, film, starts, with, a, manager, nicholas...","[film, starts, manager, nicholas, bell, giving..."
3,3630_4,0,it must be assumed that those who praised this...,1,0,0,1,"[it, must, be, assumed, that, those, who, prai...","[must, assumed, praised, film, greatest, filme..."
4,9495_8,1,superbly trashy and wondrously unpretentious 8...,0,0,1,1,"[superbly, trashy, and, wondrously, unpretenti...","[superbly, trashy, wondrously, unpretentious, ..."


4. Apply NLTK’s PorterStemmer.

In [115]:
#load library
from nltk.stem.porter import PorterStemmer

#create stemmer
porter = PorterStemmer()

#create function to stem words
def stem_words(text):
    text = [porter.stem(word) for word in text]
    listToStr = ' '.join([str(elem) for elem in text])

    return listToStr

#apply function to reviews
reviews_train['apply_stemmer'] = reviews_train['removed_stopwords'].apply(lambda x: stem_words(x))


5. Create a bag-of-words matrix from your stemmed text (output from (4)), where each row is a word-count vector for a single movie review (see sections 5.3 & 6.8 in the Machine Learning with Python Cookbook). Display the dimensions of your bag-of-words matrix. The number of rows in this matrix should be the same as the number of rows in your original data frame.

In [116]:
#load library
from sklearn.feature_extraction.text import CountVectorizer

num_reviews = reviews_train["apply_stemmer"].size

#Don't need this, but part of tutorial
#clean_train_reviews = []
#for i in range(0, num_reviews):                                                              
#    clean_train_reviews.append(reviews_train["apply_stemmer"][i] )

#initialize vector
count = CountVectorizer()

#Apply to dataset
train_data_features = count.fit_transform(reviews_train['apply_stemmer'])

#turn into array
train_data_features = train_data_features.toarray()

#look at dimensions, confirm same # rows
print(train_data_features.shape)
#print(text_data)
#bag_of_words = count.fit_transform(text_data)

#bag_of_words.toarray()

(25000, 51216)


In [117]:
#check features 
vocab = count.get_feature_names()

# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
#for tag, count in zip(vocab, dist):
    #print(count, tag)

6. Create a term frequency-inverse document frequency (tf-idf) matrix from your stemmed text, for your movie reviews (see section 6.9 in the Machine Learning with Python Cookbook). Display the dimensions of your tf-idf matrix. These dimensions should be the same as your bag-of-words matrix.

In [40]:
#load library
from sklearn.feature_extraction.text import TfidfVectorizer

#create the tf-idf matrix
tfidf = TfidfVectorizer()
feature_matrix = tfidf.fit_transform(reviews_train['apply_stemmer'])

#show the tf-idf feature matrix
feature_matrix

#get the most common words
tfidf.vocabulary_

print(feature_matrix)

#Same as bag-of-words

KeyError: 'apply_stemmer'

In [42]:
print(feature_matrix)

NameError: name 'feature_matrix' is not defined